# Claudia Pinzón

## Taller #11: Twitter

###  `[25 pts]` Punto 1: Extraer Tuits
Extraer mínimo 100 tuits de Twitter usando la API

❌ No escoger un tema de los que hicimos en clase (día de las madres, Giro de Italia, #EraGoldeYepes)

In [1]:
import pandas as pd
import tweepy
import json

In [2]:
#Leer las llaves 
keys = pd.read_csv("C:/Users/eliza/OneDrive/Desktop/Claus/Konrad/NLP/NLPtwitter.csv", header=None)
keys = dict(zip(keys[0], keys[1]))
keys.keys()

dict_keys(['api_key', 'api_secret_key', 'access_token', 'access_token_secret', 'bearer_token'])

In [3]:
# Autenticar a través de la API 
auth = tweepy.OAuthHandler(keys['api_key'], keys['api_secret_key'])
auth.set_access_token(keys['access_token'], keys['access_token_secret'])
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
tweets = tweepy.Cursor(api.search,
                      q=['migrantes colombianos -RT'],
                      lang='es',
                      geocode='-35.67515,-71.54297,248km',#Chile
                      tweet_mode = 'extended',
                      include_rts=False
                      ).items(100)

In [5]:
# Objeto generador
t = [tweet for tweet in tweets]
len(t)

3

In [6]:
import snscrape.modules.twitter as sntwitter

In [7]:
datos = []

maxTweets = 200  
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('migrantes colombianos '+'since:2011-01-01 until:2014-12-31').get_items()) :
        if i > maxTweets :
            break
        tw=[]
        tw.append(tweet.content)
        tw.append(tweet.date)
        tw.append(tweet)
        
        datos.append(tw)

datos = pd.DataFrame(datos)
datos.columns = ['tweets', 'date', 'links']
datos.head()

,tweets,date,links
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,(https://twitter.com/fjnicolas/status/54899566...
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,(https://twitter.com/angelcalleja/status/54556...
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,(https://twitter.com/Colombiaenmi/status/54555...
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,(https://twitter.com/ConnectasOrg/status/54536...
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,(https://twitter.com/ConnectasLab/status/54536...


In [8]:
len(datos)

201

In [9]:
#Quitar links, menciones y hashtag
import re

In [10]:
pattern1 = '?P<pic>pic.twitter.com/[^\s]+'
pattern2 = '?P<url>https?://[^\s]+'

def text_clean(row):
    text = row['tweets'] 
    
    links = [tuple(j for j in i if j)[-1] for i in re.findall(f"({pattern1})|({pattern2})",text)]
    for link in links:
        text = text.replace(link,"")
    
    hashtags = [interaction for interaction in text.split() if interaction.startswith("#")]
    for hashtag in hashtags:
        text = text.replace(hashtag, "")
        
    mentions = [interaction for interaction in text.split() if interaction.startswith("@")]
    for mention in mentions:
        text = text.replace(mention, "")
        
    return text, links, hashtags, mentions

In [11]:
datos[['texto_limpio', 'links', 'hashtags', 'mentions']] = datos.apply(text_clean, axis=1, result_type='expand')
datos

,tweets,date,links,texto_limpio,hashtags,mentions
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,[],Uno de los puntajes nacionales es hijo de migr...,[],[]
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,[],"Es que si no hay una guerra nueva, negros famé...",[],[]
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,[http://t.co/L4wSw2jIAY],"Migrantes nuestro dia, nuestra oportunidad CO...",[#tambiendecidimos],[]
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,[http://t.co/6Na8byvdVZ],Migrantes colombianos que llegaron a Antofagas...,[#NuevoÉxodoLatino],[]
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,[http://t.co/cgzBdDL1Xj],Migrantes colombianos que llegaron a Antofagas...,[#NuevoÉxodoLatino],[]
...,...,...,...,...,...,...
196,Los colombianos migrantes necesitan un preside...,2014-04-26 19:31:16+00:00,[http://t.co/KaSoCspsB4],Los colombianos migrantes necesitan un preside...,[#BlogSemana],[@EnriquePenalosa]
197,@ameliarueda hay prioridades mas grandes que ...,2014-04-25 03:50:51+00:00,[],hay prioridades mas grandes que las peleas d...,[],[@ameliarueda]
198,MINTRABAJO IMPLEMENTARÁ MEDIDAS DE PROTECCIÓN ...,2014-04-22 21:59:34+00:00,[http://t.co/IfRfhurlL4],MINTRABAJO IMPLEMENTARÁ MEDIDAS DE PROTECCIÓN ...,[],[]
199,Diseñamos esta GUIA PARA MIGRANTES COLOMBIANOS...,2014-04-15 16:18:40+00:00,[http://t.co/rSIhtX3ts9],Diseñamos esta GUIA PARA MIGRANTES COLOMBIANOS...,[],[]


In [12]:
#Quitar emojis
import regex
import emoji

In [13]:
def get_emojis(text):
    emoji_list = []
    datos = regex.findall(r'\X', text)
    for word in datos:
        if any(char in emoji.UNICODE_EMOJI['en'] for char in word):
            emoji_list.append(word)
    
    return emoji_list

In [14]:
datos['emojis'] = datos['texto_limpio'].apply(lambda text: get_emojis(text))
datos.head()

,tweets,date,links,texto_limpio,hashtags,mentions,emojis
0,Uno de los puntajes nacionales es hijo de migr...,2014-12-28 00:15:41+00:00,[],Uno de los puntajes nacionales es hijo de migr...,[],[],[]
1,"Es que si no hay una guerra nueva, negros famé...",2014-12-18 13:05:45+00:00,[],"Es que si no hay una guerra nueva, negros famé...",[],[],[]
2,"Migrantes nuestro dia, nuestra oportunidad #ta...",2014-12-18 12:39:38+00:00,[http://t.co/L4wSw2jIAY],"Migrantes nuestro dia, nuestra oportunidad CO...",[#tambiendecidimos],[],[]
3,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:42+00:00,[http://t.co/6Na8byvdVZ],Migrantes colombianos que llegaron a Antofagas...,[#NuevoÉxodoLatino],[],[]
4,Migrantes colombianos que llegaron a Antofagas...,2014-12-17 23:30:39+00:00,[http://t.co/cgzBdDL1Xj],Migrantes colombianos que llegaron a Antofagas...,[#NuevoÉxodoLatino],[],[]


In [15]:
#Serie de Tiempo

datos['hora'] = datos['date'].dt.floor('T').dt.time #Hora y minuto quitando segundos, conversión en formato "tiempo"
temp = pd.DataFrame(datos.hora.value_counts()).reset_index() #convertir la información en dataframe
temp.columns = ['hora', 'cnt'] #convertir en columna
temp = temp.sort_values('hora') #convertirlo en dataframe temporal
temp

,hora,cnt
156,00:05:00,1
135,00:10:00,1
164,00:13:00,1
7,00:15:00,2
120,00:20:00,1
...,...,...
44,23:31:00,1
40,23:33:00,1
106,23:46:00,1
115,23:50:00,1


In [16]:
import plotly.io as pio
import plotly.express as px
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import matplotlib.pyplot as plt
%matplotlib inline
init_notebook_mode(connected=True)

In [17]:
trace = go.Scatter(x=temp.hora.values,
                   y=temp.cnt.values,
                   text=[f"Fecha: {f}<br>Tuits: {c}" for f,c in zip(temp.hora.values,temp.cnt.values)],
                   hoverinfo='text',
                   mode='lines+markers',
                   name='Horas',
                   line={'color': 'blue'})

layout = go.Layout(title="Número de tweets por hora")

fig = go.Figure(data=[trace], layout=layout)
iplot(fig)

###  `[25 pts]` Punto 2: Hacer una visualización sobre esos tuits

In [18]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
stopwords_sp = stopwords.words('spanish')

In [19]:
#Vectorización

def pre_procesado(texto):
    texto = texto.lower()
    texto = re.sub(r"[\W\d_´]+", " ", texto)
    texto = " ".join([palabra for palabra in texto.split() if palabra not in stopwords_sp])
    return (texto)

tfidf_vect = TfidfVectorizer(preprocessor=pre_procesado)
tfidf = tfidf_vect.fit_transform(datos.texto_limpio.values)

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

voc = [k for k,v in sorted(tfidf_vect.vocabulary_.items(), key=lambda kv: kv[1])]
temp = pd.DataFrame(tfidf.toarray())
temp = cosine_similarity(temp.values)
temp = pd.DataFrame(temp)
temp

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
0,1.000000,0.008323,0.017124,0.012548,0.012548,0.010714,0.014841,0.011601,0.010475,0.007759,...,0.017747,0.017747,0.009424,0.010630,0.010079,0.011954,0.009234,0.012037,0.012693,0.016037
1,0.008323,1.000000,0.013229,0.009694,0.009694,0.008277,0.011465,0.008962,0.008092,0.005994,...,0.013710,0.013710,0.007281,0.008212,0.007786,0.009235,0.007134,0.009299,0.009806,0.012389
2,0.017124,0.013229,1.000000,0.019944,0.019944,0.017029,0.023589,0.018439,0.016649,0.012332,...,0.028207,0.028207,0.014979,0.016896,0.016019,0.019001,0.014678,0.019132,0.020174,0.025490
3,0.012548,0.009694,0.019944,1.000000,1.000000,0.012479,0.017285,0.013511,0.012200,0.009037,...,0.020670,0.020670,0.010977,0.012381,0.011739,0.013923,0.010755,0.014019,0.014783,0.018678
4,0.012548,0.009694,0.019944,1.000000,1.000000,0.012479,0.017285,0.013511,0.012200,0.009037,...,0.020670,0.020670,0.010977,0.012381,0.011739,0.013923,0.010755,0.014019,0.014783,0.018678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.011954,0.009235,0.019001,0.013923,0.013923,0.011889,0.016468,0.012872,0.011623,0.008609,...,0.019692,0.019692,0.010458,0.011795,0.158506,1.000000,0.010247,0.013356,0.014084,0.017795
197,0.009234,0.007134,0.014678,0.010755,0.010755,0.009184,0.012721,0.009943,0.008979,0.006650,...,0.015211,0.015211,0.008078,0.009112,0.008639,0.010247,1.000000,0.010317,0.010880,0.013746
198,0.012037,0.009299,0.019132,0.014019,0.014019,0.011971,0.016581,0.012961,0.011703,0.008669,...,0.260630,0.260630,0.010530,0.011877,0.011261,0.013356,0.010317,1.000000,0.014181,0.235522
199,0.012693,0.009806,0.020174,0.014783,0.014783,0.012623,0.017485,0.013667,0.012341,0.009141,...,0.020908,0.020908,0.011103,0.012524,0.011874,0.014084,0.010880,0.014181,1.000000,0.018894


In [21]:
import scipy.cluster.hierarchy as sch
import numpy as np

In [22]:
pairwise_distances = sch.distance.pdist(temp)
linkage = sch.linkage(pairwise_distances, method='ward')
idx_to_cluster_array = sch.fcluster(linkage, pairwise_distances.max() * 0.5, criterion='distance')
idx = np.argsort(idx_to_cluster_array)
temp = temp.copy()
    
temp2 = temp.iloc[idx, :].T.iloc[idx, :]
my_idx = idx_to_cluster_array

temp2.index = ['tweet #'+str(i) for i in temp2.columns]
temp2.columns = temp2.index

In [23]:
#Gráfico

trace = go.Heatmap(z=temp2.values.tolist(),
                   x=temp2.index.values,
                   y=temp2.columns.values,
                   colorscale='Greens')

layout = go.Layout(title='Mapa de Calor: Tweets 2011-2014 en Chile, Keyword "Migrantes colombianos"',
                   width=705, height=600)

fig = go.Figure(data=[trace],layout=layout)
iplot(fig)

In [24]:
datos['cluster'] = my_idx 

for c in datos.cluster.value_counts().index:
    df_temp = datos[datos.cluster==c]
    
    sim = temp2[['tweet #'+str(i) for i in df_temp.index]].mean().mean()
    print(f"***** El cluster {c} tiene {df_temp.shape[0]} tweets con una similaridad de {sim:.2%} *****\n")
    
    tweets_ejemplo = df_temp.head().tweets.values
    for te in tweets_ejemplo:
        print("tweet")
        print(te)
        print()
    print()

***** El cluster 16 tiene 92 tweets con una similaridad de 2.48% *****

tweet
Uno de los puntajes nacionales es hijo de migrantes colombianos... ahí tienen chilenazis de mierda!

tweet
Es que si no hay una guerra nueva, negros famélicos, invasiones por la valla, o colombianos trayendo coca, el día de los migrantes no mola.

tweet
Migrantes nuestro dia, nuestra oportunidad #tambiendecidimos COMO colombianos en el exterior http://t.co/L4wSw2jIAY

tweet
Comprobado: colombianos atienden mejor y con alegría en #VegaDeAntofagasta. Ese es un gran aporte de migrantes a Chile #ChileMulticultural

tweet
"Foro abierto a colombianos migrantes o exiliados colombianos /as víctimas d violencia política, social y económica" http://t.co/6Ih8lVRyI0


***** El cluster 6 tiene 16 tweets con una similaridad de 10.24% *****

tweet
Migrantes colombianos en #Europa, desafíos después de la asamblea http://t.co/SQ0A6cnJEG http://t.co/aAidWGkhdq

tweet
#Internacional | Migrantes colombianos en Europa, desafíos d